# 1. Tokenization

## 1-1. Word Tokenization(영어)

- 단어 단위로 토큰화를 수행하는 것을 Word Tokenization이라고 합니다.

In [64]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from konlpy.tag import *
from ckonlpy.tag import Twitter
import MeCab
import kss
from collections import Counter
import tensorflow as tf
import numpy as np
import re
from soynlp.normalizer import *
import hanspell

okt = Okt()
kkm = Kkma()
kmr = Komoran()
hnn = Hannanum()
twt = Twitter()

class Mecab:
    def pos(self, text):
        p = re.compile(".+\t[A-Z]+")
        return [tuple(p.match(line).group().split("\t")) for line in MeCab.Tagger().parse(text).splitlines()[:-1]]
    
    def morphs(self, text):
        p = re.compile(".+\t[A-Z]+")
        return [p.match(line).group().split("\t")[0] for line in MeCab.Tagger().parse(text).splitlines()[:-1]]
    
    def nouns(self, text):
        p = re.compile(".+\t[A-Z]+")
        temp = [tuple(p.match(line).group().split("\t")) for line in MeCab.Tagger().parse(text).splitlines()[:-1]]
        nouns=[]
        for word in temp:
            if word[1] in ["NNG", "NNP", "NNB", "NNBC", "NP", "NR"]:
                nouns.append(word[0])
        return nouns
    
mc = Mecab()

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\5CG7092POZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\5CG7092POZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\5CG7092POZ\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### (1) word_tokenize()
- 아포스트로피가 들어간 상황에서 Don"t와 Jone"s는 어떻게 토큰화할 수 있을까요?

In [3]:
sentence = "Don't be fooled by the dark sounding name, Mr. Jone's Orphanage is as cheery as cheery goes for a pastry shop."

print(word_tokenize(sentence))

['Do', "n't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr.', 'Jone', "'s", 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']


- Don"t를 Do와 n"t로 분리하였으며, Jone"s는 Jone과 "s로 분리

### (2) WordPunctTokenizer().tokenize()

In [4]:
print(WordPunctTokenizer().tokenize(sentence))

['Don', "'", 't', 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', ',', 'Mr', '.', 'Jone', "'", 's', 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop', '.']


- Don"t를 Don과 "와 t로 분리하였으며, Jone"s를 Jone과 "와 s로 분리
- 정답은 없습니다. 사실 토크나이저마다 각자 규칙이 다르기 때문에 사용하고자 하는 목적에 따라 토크나이저를 선택하는 것이 중요합니다.

### (3) TreebankWordTokenizer()
- Penn Treebank Tokenization 규칙
    - 규칙 1. 하이푼으로 구성된 단어는 하나로 유지한다.
    - 규칙 2. doesn"t와 같이 아포스트로피로 "접어"가 함께하는 단어는 분리해준다. 

In [17]:
text = "Starting a home-based restaurant may be an ideal. it doesn"t have a food chain or restaurant of their own."

print(TreebankWordTokenizer().tokenize(text))

['Starting', 'a', 'home-based', 'restaurant', 'may', 'be', 'an', 'ideal.', 'it', 'does', "n't", 'have', 'a', 'food', 'chain', 'or', 'restaurant', 'of', 'their', 'own', '.']


## 1-2. Word Tokenization(한국어)

- 한국어의 특성으로 인해 영어와는 토큰화가 좀 더 까다롭습니다.  
- 영어의 경우에는 사실 띄어쓰기로 토큰화를 하더라도 큰 문제가 되지 않는 경우가 많은데 한국어의 경우 어절(띄어쓰기 단위) 토큰화는 지양합니다.
- 위 형태소 분석기들은 공통적으로 아래의 함수를 제공합니다.  
    - nouns : 명사 추출  
    - morphs : 형태소 추출  
    - pos : 품사 부착

### (1) Okt()

In [181]:
text = "열심히 코딩한 당신, 연휴에는 여행을 가봐요"

print(okt.nouns(text))
print(okt.morphs(text))
print(okt.pos(text))

['코딩', '당신', '연휴', '여행']
['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']
[('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('가봐요', 'Verb')]


### (2) Kkma()

In [182]:
print(kkm.nouns(text))
print(kkm.morphs(text))
print(kkm.pos(text))

['코딩', '당신', '연휴', '여행']
['열심히', '코딩', '하', 'ㄴ', '당신', ',', '연휴', '에', '는', '여행', '을', '가보', '아요']
[('열심히', 'MAG'), ('코딩', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('당신', 'NP'), (',', 'SP'), ('연휴', 'NNG'), ('에', 'JKM'), ('는', 'JX'), ('여행', 'NNG'), ('을', 'JKO'), ('가보', 'VV'), ('아요', 'EFN')]


### (3) Komoran()

In [183]:
print(kmr.nouns(text))
print(kmr.morphs(text))
print(kmr.pos(text))

['코', '당신', '연휴', '여행']
['열심히', '코', '딩', '하', 'ㄴ', '당신', ',', '연휴', '에', '는', '여행', '을', '가', '아', '보', '아요']
[('열심히', 'MAG'), ('코', 'NNG'), ('딩', 'MAG'), ('하', 'XSV'), ('ㄴ', 'ETM'), ('당신', 'NNP'), (',', 'SP'), ('연휴', 'NNG'), ('에', 'JKB'), ('는', 'JX'), ('여행', 'NNG'), ('을', 'JKO'), ('가', 'VV'), ('아', 'EC'), ('보', 'VX'), ('아요', 'EC')]


### (4) Hannanum()

In [184]:
print(hnn.nouns(text))
print(hnn.morphs(text))
print(hnn.pos(text))

['코딩', '당신', '연휴', '여행']
['열심히', '코딩', '하', 'ㄴ', '당신', ',', '연휴', '에는', '여행', '을', '가', '아', '보', '아']
[('열심히', 'M'), ('코딩', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('당신', 'N'), (',', 'S'), ('연휴', 'N'), ('에는', 'J'), ('여행', 'N'), ('을', 'J'), ('가', 'P'), ('아', 'E'), ('보', 'P'), ('아', 'E')]


### (5) MeCab

In [187]:
print(mc.nouns(text))
print(mc.morphs(text))
print(mc.pos(text))

['코딩', '당신', '연휴', '여행']
['열심히', '코딩', '한', '당신', ',', '연휴', '에', '는', '여행', '을', '가', '봐요']
[('열심히', 'MAG'), ('코딩', 'NNG'), ('한', 'XSA'), ('당신', 'NP'), (',', 'SC'), ('연휴', 'NNG'), ('에', 'JKB'), ('는', 'JX'), ('여행', 'NNG'), ('을', 'JKO'), ('가', 'VV'), ('봐요', 'EC')]


각 형태소 분석기는 성능과 결과가 다르게 나오기 때문에, 형태소 분석기의 선택은 사용하고자 하는 필요 용도에 어떤 형태소 분석기가 가장 적절한지를 판단하고 사용하면 됩니다. 예를 들어서 속도를 중시한다면 메캅을 사용할 수 있습니다.



## 2-1. Sentence Tokenization(영어)
- 직관적으로 생각해봤을 때는 ?나 온점(.)이나 ! 기준으로 문장을 잘라내면 되지 않을까라고 생각할 수 있지만, 꼭 그렇지만은 않습니다. !나 ?는 문장의 구분을 위한 꽤 명확한 구분자(boundary) 역할을 하지만 온점은 꼭 그렇지 않기 때문입니다. 다시 말해, 온점은 문장의 끝이 아니더라도 등장할 수 있습니다.
    - IP 192.168.56.31 서버에 들어가서 로그 파일 저장해서 ukairia777@gmail.com로 결과 좀 보내줘. 그러고나서 점심 먹으러 가자.
    - I am actively looking for Ph.D. students and you are a Ph.D student.
- 온점을 기준으로 문장을 구분할 경우에는 예외사항이 너무 많습니다.  
- NLTK에서는 영어 문장의 토큰화를 수행하는 sent_tokenize를 지원하고 있습니다.

### (1) sent_tokenize()

In [33]:
text = "His barber kept his word. But keeping such a huge secret to himself was driving him crazy. Finally, the barber went up a mountain and almost to the edge of a cliff. He dug a hole in the midst of some reeds. He looked about, to mae sure no one was near."

In [12]:
print(sent_tokenize(text))

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [5]:
text="I am actively looking for Ph.D. students and you are a Ph.D student."

print(sent_tokenize(text))

['I am actively looking for Ph.D. students and you are a Ph.D student.']


## 2-2. Sentence Tokenization(한국어)

In [8]:
text = "딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어려워요. 농담아니에요. 이제 해보면 알걸요?"

print(kss.split_sentences(text))

['딥 러닝 자연어 처리가 재미있기는 합니다.', '그런데 문제는 영어보다 한국어로 할 때 너무 어려워요.', '농담아니에요.', '이제 해보면 알걸요?']


### 한국어 띄어쓰기 패키지(pykospacing)
- 한국어의 경우 띄어쓰기가 잘 지켜지지 않는다. 띄어쓰기 패키지를 통해서 띄어쓰기가 되어있지 않은 문장을 보정해보자.

In [55]:
import pykospacing

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [56]:
sent = "오지호는 극중 두 얼굴의 사나이 성준 역을 맡았다. 성준은 국내 유일의 태백권 전승자를 가리는 결전의 날을 앞두고 20년간 동고동락한 사형인 진수(정의욱 분)를 찾으러 속세로 내려온 인물이다."
#띄어쓰기가 없는 문장 임의로 만듭니다.
new_sent = sent.replace(" ", "")

print(new_sent)

오지호는극중두얼굴의사나이성준역을맡았다.성준은국내유일의태백권전승자를가리는결전의날을앞두고20년간동고동락한사형인진수(정의욱분)를찾으러속세로내려온인물이다.


In [57]:
sent_ks = pykospacing.spacing(new_sent)

print(sent)
print(sent_ks)

NameError: name 'pykospacing' is not defined

# 2. Text Normalization

- 텍스트 정규화(Text Normalization)은 통일할 수 있는 단어들은 하나로 통일하기 위한 전처리입니다. 보통 정규 표현식이나 자신만의 규칙을 정의할 수도 있지만 Stemming이나 Lemmatization도 사용합니다.

## 2-1. Stemming

- 어간(Stem)을 추출하는 작업을 어간 추출(stemming)이라고 합니다. 어간 추출은 형태학적 분석을 단순화한 버전이라고 볼 수도 있고, 정해진 규칙만 보고 단어의 어미를 자르는 어림짐작의 작업이라고 볼 수도 있습니다. 다시 말해, 이 작업은 섬세한 작업이 아니기 때문에 어간 추출 후에 나오는 결과 단어는 사전에 존재하지 않는 단어일 수도 있습니다.
- 가령, 포터 스테머의 포터 알고리즘의 어간 추출은 이러한 규칙들을 가집니다.  
    - ALIZE → AL  
    - ANCE → 제거  
    - ICAL → IC  

In [60]:
ps = PorterStemmer()

text = "This was not the map we found in Billy Bones"s chest, but an accurate copy, complete in all things--names and heights and soundings--with the single exception of the red crosses and the written notes."
words = word_tokenize(text)

print(words)

In [62]:
print([ps.stem(word) for word in words])

['thi', 'wa', 'not', 'the', 'map', 'we', 'found', 'in', 'billi', 'bone', "'s", 'chest', ',', 'but', 'an', 'accur', 'copi', ',', 'complet', 'in', 'all', 'thing', '--', 'name', 'and', 'height', 'and', 'sound', '--', 'with', 'the', 'singl', 'except', 'of', 'the', 'red', 'cross', 'and', 'the', 'written', 'note', '.']


- 조금 더 간단한 예제로 확인해봅시다.

In [63]:
words = ["formalize", "allowance", "electricical"]

print([ps.stem(word) for word in words])

['formal', 'allow', 'electric']


## 2-2. Lemmatization

- 표제어 추출은 단어들이 다른 형태를 가지더라도, 그 뿌리 단어를 찾아가서 단어의 개수를 줄일 수 있는지 판단합니다. 예를 들어서 am, are, is는 서로 다른 스펠링이지만 그 뿌리 단어는 be라고 볼 수 있습니다. 이 때, 이 단어들의 표제어는 be라고 합니다.

In [56]:
wnl = WordNetLemmatizer()
words = ["policy", "doing", "organization", "have", "going", "love", "lives", "fly", "dies", "watched", "has", "starting"]

print([wnl.lemmatize(word) for word in words])

['policy', 'doing', 'organization', 'have', 'going', 'love', 'life', 'fly', 'dy', 'watched', 'ha', 'starting']


- 위의 결과에서는 dy나 ha와 같이 의미를 알 수 없는 적절하지 못한 단어를 출력하고 있습니다. 이는 표제어 추출기(lemmatizer)가 본래 단어의 품사 정보를 알아야만 정확한 결과를 얻을 수 있기 때문입니다.

In [59]:
wnl.lemmatize("watched", "v")

'watch'

## Stemming and Normalization(한국어)

In [3]:
text = "북한은 하루새 3차례에 걸쳐 대미·대남 압박 메시지를 내놓았다."

print(okt.morphs(text))
print(okt.morphs(text, stem=True))
print(okt.pos(text))
print(okt.pos(text, stem=True))

['북한', '은', '하루', '새', '3', '차례', '에', '걸쳐', '대미', '·', '대남', '압박', '메시지', '를', '내놓았다', '.']
['북한', '은', '하루', '새', '3', '차례', '에', '걸치다', '대미', '·', '대남', '압박', '메시지', '를', '내놓다', '.']
[('북한', 'Noun'), ('은', 'Josa'), ('하루', 'Noun'), ('새', 'Noun'), ('3', 'Number'), ('차례', 'Noun'), ('에', 'Josa'), ('걸쳐', 'Verb'), ('대미', 'Noun'), ('·', 'Punctuation'), ('대남', 'Noun'), ('압박', 'Noun'), ('메시지', 'Noun'), ('를', 'Josa'), ('내놓았다', 'Verb'), ('.', 'Punctuation')]
[('북한', 'Noun'), ('은', 'Josa'), ('하루', 'Noun'), ('새', 'Noun'), ('3', 'Number'), ('차례', 'Noun'), ('에', 'Josa'), ('걸치다', 'Verb'), ('대미', 'Noun'), ('·', 'Punctuation'), ('대남', 'Noun'), ('압박', 'Noun'), ('메시지', 'Noun'), ('를', 'Josa'), ('내놓다', 'Verb'), ('.', 'Punctuation')]


In [5]:
text = "웃기는 소리하지마랔ㅋㅋㅋ"

print(okt.morphs(text))
print(okt.morphs(text, norm=True))
print(okt.pos(text))
print(okt.pos(text, norm=True))

['웃기는', '소리', '하지마', '랔', 'ㅋㅋㅋ']
['웃기는', '소리', '하지마라', 'ㅋㅋㅋ']
[('웃기는', 'Verb'), ('소리', 'Noun'), ('하지마', 'Verb'), ('랔', 'Noun'), ('ㅋㅋㅋ', 'KoreanParticle')]
[('웃기는', 'Verb'), ('소리', 'Noun'), ('하지마라', 'Verb'), ('ㅋㅋㅋ', 'KoreanParticle')]


### 한국어 Norm : 반복되는 문자 정제(soynlp)
- ㅋㅋ, ㅎㅎ 등의 이모티콘의 경우 불필요하게 연속되는 경우가 많은데 ㅋㅋ, ㅋㅋㅋ, ㅋㅋㅋㅋ와 같은 경우를 모두 서로 다른 단어로 처리하는 것은 불필요합니다. 이에 반복되는 것은 하나로 정규화시켜주는 것이 좋습니다.

In [60]:
print(emoticon_normalize("앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ", num_repeats=2))
print(emoticon_normalize("앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠ", num_repeats=2))
print(emoticon_normalize("앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ", num_repeats=2))
print(emoticon_normalize("앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ", num_repeats=2))

아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ
아ㅋㅋ영화존잼쓰ㅠㅠ


- 의미없게 반복되는 것은 비단 이모티콘에 한정되지 않습니다.

In [61]:
print(repeat_normalize("와하하하하하하하하하핫", num_repeats=2))
print(repeat_normalize("와하하하하하하핫", num_repeats=2))
print(repeat_normalize("와하하하하핫", num_repeats=2))

와하하핫
와하하핫
와하하핫


### 한국어 Norm : 맞춤법 교정(hanspell)

In [66]:
spelled_sent

Checked(result=True, original='맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지', checked='맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지', errors=2, words=OrderedDict([('맞춤법', 0), ('틀리면', 0), ('왜', 1), ('안돼?', 1), ('쓰고', 1), ('싶은', 1), ('대로', 1), ('쓰면', 1), ('되지', 1)]), time=0.03889656066894531)

In [67]:
hanspell.spell_checker.check(sent).checked

'맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지'

In [68]:
sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지"
sent_ckd = hanspell.spell_checker.check(sent).checked

print(sent_ckd)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


- hanspell은 사실 띄어쓰기 교정 또한 하고 있다.

In [70]:
sent = "오지호는극중두얼굴의사나이성준역을맡았다.성준은국내유일의태백권전승자를가리는결전의날을앞두고20년간동고동락한사형인진수(정의욱분)를찾으러속세로내려온인물이다."
sent_ckd = hanspell.spell_checker.check(sent).checked

print(sent_ckd)
# print(sent_ks)

오지호는 극 중 두 얼굴의 사나이 성준 역을 맡았다. 성준은 국내 유일의 태백권 전승자를 가리는 결전의 날을 앞두고 20년간 동고동락한 사형인 진수(정인욱 분)를 찾으러 속세로 내려온 인물이다.


In [74]:
sent_ckd.split(" ")

['오지호는',
 '극',
 '중',
 '두',
 '얼굴의',
 '사나이',
 '성준',
 '역을',
 '맡았다.',
 '성준은',
 '국내',
 '유일의',
 '태백권',
 '전승자를',
 '가리는',
 '결전의',
 '날을',
 '앞두고',
 '20년간',
 '동고동락한',
 '사형인',
 '진수(정인욱',
 '분)를',
 '찾으러',
 '속세로',
 '내려온',
 '인물이다.']

# 3. Stopwords

- 영어에서의 I, my, me, over, the와 같은 단어들이나 한국어의  조사, 접미사 같은 단어들은 문장에서는 자주 등장하지만 실제 의미 분석을 하는데는 거의 기여하는 바가 없는 경우가 있습니다. 이러한 단어들을 불용어(stopword)라고 하며, NLTK에서는 위와 같은 100여개 이상의 영어 단어들을 불용어로 패키지 내에서 미리 정의하고 있습니다.

In [122]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [75]:
example = "Family is not an important thing. It's everything."
stop_words = set(stopwords.words('english')) 

word_tokens = word_tokenize(example)

result = []
for w in word_tokens: 
    if w not in stop_words: 
        result.append(w) 

print('원문 :', word_tokens) 
print('불용어 제거 후 :', result) 

원문 : ['Family', 'is', 'not', 'an', 'important', 'thing', '.', 'It', "'s", 'everything', '.']
불용어 제거 후 : ['Family', 'important', 'thing', '.', 'It', "'s", 'everything', '.']


In [76]:
example = "와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔"
word_tokens = okt.morphs(example)
print(word_tokens)

['와', '이런', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만드는', '게', '나을', '뻔']


In [77]:
stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '것']
# 위의 불용어는 저자가 임의로 선정한 것으로 실제 의미있는 선정 기준이 아님

result=[word for word in word_tokens if not word in stop_words]

print('원문 :', word_tokens) 
print('불용어 제거 후 :', result) 

원문 : ['와', '이런', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만드는', '게', '나을', '뻔']
불용어 제거 후 : ['이런', '영화', '라고', '차라리', '뮤직비디오', '만드는', '게', '나을', '뻔']


# 4. Vocabulary

In [4]:
text = "A barber is a person. a barber is good person. a barber is huge person. \
        he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. \
        His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. \
        the barber went up a huge mountain."

corpus = []
for sent in sent_tokenize(text):
    words = []
    for word in word_tokenize(sent):
        word = word.lower()
        if (word not in stopwords.words("english")) & (word not in [".", "!"]):
            words.append(word)
    corpus.append(words)

words = sum(corpus, [])

print(corpus)
print(words)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]
['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


- 단어 집합을 생성합니다. 단어 집합이란 중복을 제거한 단어들의 집합을 의미합니다.

In [5]:
vocab = Counter(words) # 파이썬의 Counter 모듈을 이용하면 단어의 모든 빈도를 쉽게 계산할 수 있습니다.
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [6]:
vocab["barber"] # "barber"라는 단어의 빈도수 출력

8

## Integer encoding(Python)

- 빈도수가 높은 순서대로 정렬하고, 빈도수가 높을 수록 낮은 수의 정수를 부여합니다.  
이렇게하면 빈도수가 낮은 단어들을 제외시키면서 단어 집합의 크기를 조절하기가 편합니다.

In [7]:
# 빈도수가 높은 순서대로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x : x[1], reverse=True)

print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


In [8]:
# 이제 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여합니다.
word2idx = {}
i=0
for word, freq in vocab_sorted :
    if freq > 1 : # 정제(Cleaning) 챕터에서 언급했듯이 빈도수가 적은 단어는 제외한다.
        i += 1
        word2idx[word] = i
        
print(word2idx)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


- 자연어 처리를 하다보면, 텍스트 데이터에 있는 단어를 모두 사용하기 보다는 빈도수가 가장 높은 n개의 단어만 사용하고 싶은 경우가 많습니다. 위 단어들은 빈도수가 높은 순으로 낮은 정수가 부여되어져 있으므로 빈도수 상위 n개의 단어만 사용하고 싶다고하면 vocab에서 정수값이 1부터 n까지인 단어들만 사용하면 됩니다. 여기서는 상위 5개 단어만 사용한다고 가정하겠습니다.

In [9]:
n_vocabs = 5
#index가 5 초과인 단어 제거
keys_pop = [k for k, v in word2idx.items() if v >= n_vocabs + 1]
for key in keys_pop:
    word2idx.pop(key)

print(word2idx)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### Out Of Vocabulary(UNK) Problem
- 이제 word2idx에는 빈도수가 높은 상위 5개의 단어만 저장되었습니다. 이제 word2idx를 사용하여 단어 토큰화가 된 상태로 저장된 corpus에 있는 각 단어를 정수로 바꾸는 작업을 하겠습니다.
- 예를 들어 corpus에서 첫번째 문장은 ["barber", "person"]이었는데, 이 문장에 대해서는 [1, 5]로 인코딩합니다. 그런데 두번째 문장인 ["barber", "good", "person"]에는 더 이상 word2idx에는 존재하지 않는 단어인 "good"이라는 단어가 있습니다.  
- 이처럼 단어 집합에 존재하지 않는 단어들을 Out-Of-Vocabulary(단어 집합에 없는 단어)의 약자로 "UNK"라고 합니다. word_to_index에 "UNK"란 단어를 새롭게 추가하고, 단어 집합에 없는 단어들은 "UNK"의 인덱스로 인코딩하겠습니다

In [10]:
word2idx["UNK"] = 0

print(word2idx)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 0}


In [11]:
encoded = []
for sent in corpus:
    temp = []
    for word in sent:
        try:
            temp.append(word2idx[word])
        except KeyError:
            temp.append(word2idx["UNK"])
    encoded.append(temp)

encoded

[[1, 5],
 [1, 0, 5],
 [1, 3, 5],
 [0, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 0],
 [1, 4, 0],
 [1, 4, 2],
 [0, 0, 3, 2, 0, 1, 0],
 [1, 0, 3, 0]]

## Build Vocab & Integer encoding(Tensorflow)

방금 진행했던 Build Vocabulary와 Integer encoding을 텐서플로우가 제공하는 도구를 사용하면 훨씬 더 편하게 할 수도 있습니다.

In [17]:
corpus

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

In [18]:
print(corpus)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


텐서플로우는 정수 인코딩을 수행하는 전처리 도구인 keras.preprocessing.text.Tokenizer를 제공합니다.

In [37]:
tkn = tf.keras.preprocessing.text.Tokenizer()

#빈도수를 기준으로 단어 집합을 생성한다.
tkn.fit_on_texts(corpus)

- fit_on_texts는 입력한 텍스트로부터 단어 빈도수가 높은 순으로 낮은 정수 인덱스를 부여하는데, 정확히 앞서 설명한 정수 인코딩 작업이 이루어진다고 보면됩니다. 각 단어에 인덱스가 어떻게 부여되었는지를 보려면, word_index를 사용합니다.

In [38]:
print(tkn.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


- 각 단어의 빈도수가 높은 순서대로 인덱스가 부여된 것을 확인할 수 있습니다. 각 단어가 카운트를 수행하였을 때 몇 개였는지를 보고자 한다면 word_counts를 사용합니다.

In [39]:
print(tkn.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


texts_to_sequences()는 입력으로 들어온 코퍼스에 대해서 각 단어를 이미 정해진 인덱스로 변환합니다.



In [41]:
tkn.texts_to_sequences(corpus)

[[1, 5],
 [1, 8, 5],
 [1, 3, 5],
 [9, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13]]

In [43]:
vocab_size = 5
#상위 5개 단어만 사용
tkn = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size+1)
tkn.fit_on_texts(corpus)

- num_words에서 +1을 더해서 값을 넣어주는 이유는 num_words는 숫자를 0부터 카운트합니다. 만약 5를 넣으면 0 ~ 4번 단어 보존을 의미하게 되므로 뒤의 실습에서 1번 단어부터 4번 단어만 남게됩니다. 그렇기 때문에 1 ~ 5번 단어까지 사용하고 싶다면 num_words에 숫자 5를 넣어주는 것이 아니라 5+1인 값을 넣어주어야 합니다.
- 실질적으로 숫자 0에 지정된 단어가 존재하지 않는데도 케라스 토크나이저가 숫자 0까지 단어 집합의 크기로 산정하는 이유는 자연어 처리에서 패딩(padding)이라는 작업 때문입니다. 이에 대해서는 뒤에 다루게 되므로 여기서는 케라스 토크나이저를 사용할 때는 숫자 0도 단어 집합의 크기로 고려해야한다고만 이해합시다.

In [44]:
print(tkn.texts_to_sequences(corpus))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


- 코퍼스에 대해서 각 단어를 이미 정해진 인덱스로 변환하는데, 상위 5개의 단어만을 사용하겠다고 지정하였으므로 1번 단어부터 5번 단어까지만 보존되고 나머지 단어들은 제거된 것을 볼 수 있습니다.  
- 케라스 토크나이저는 기본적으로 단어 집합에 없는 단어인 UNK에 대해서는 단어를 정수로 바꾸는 과정에서 아예 단어를 제거한다는 특징이 있습니다. 단어 집합에 없는 단어들은 UNK로 간주하여 보존하고 싶다면 Tokenizer의 인자 UNK_token을 사용합니다.
- 기본적으로 "UNK"의 인덱스를 1로 합니다.

In [47]:
vocab_size = 5
#빈도수 상위 5개 단어만 사용. 숫자 0과 UNK를 고려해서 단어 집합의 크기는 +2
tkn = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size+2, UNK_token="UNK")
tkn.fit_on_texts(corpus)

print(tkn.texts_to_sequences(corpus))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


- 빈도수 상위 5개의 단어는 2 ~ 6까지의 인덱스를 가졌으며, 그 외 단어 집합에 없는 "good"과 같은 단어들은 전부 "UNK"의 인덱스인 1로 인코딩되었습니다.

# ?. Padding

- 텍스트 데이터에 대해서 정수 인코딩을 수행하고나면, 이제 각 텍스트는 정수 시퀀스로 변환된 상태입니다. 정수 시퀀스로 변환된 각 문장(또는 문서)는 서로 길이가 다를 수 있습니다. 그런데 기계는 길이가 전부 동일한 문서들에 대해서는 하나의 행렬로 보고, 한꺼번에 묶어서 처리할 수 있습니다. 다시 말해 병렬 연산을 위해서 각 문서의 길이를 동일하게 맞춰주는 작업이 필요할 때가 있습니다.

## ?-1. Numpy로 구현하기

In [50]:
corpus

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

In [81]:
tkn.fit_on_texts(corpus)
encoded = tkn.texts_to_sequences(corpus)

In [82]:
encoded

[[1, 5],
 [1, 8, 5],
 [1, 3, 5],
 [9, 2],
 [2, 4, 3, 2],
 [3, 2],
 [1, 4, 6],
 [1, 4, 6],
 [1, 4, 2],
 [7, 7, 3, 2, 10, 1, 11],
 [1, 12, 3, 13]]

In [72]:
#가장 길이가 긴 문장의 길이를 계산해보겠습니다.
max_len = max([len(item) for item in encoded])

print(max_len)

7


- 가장 길이가 긴 문장의 길이는 7입니다. 이제 모든 문장의 길이를 7로 맞춰주겠습니다. 이때 가상의 단어 "PAD"를 사용합니다. "PAD"라는 단어가 있다고 가정하고, 이 단어는 0번 단어라고 정의해보겠습니다. 이제 길이가 7보다 짧은 문장에는 숫자 0을 채워서 전부 길이 7로 맞춰주겠습니다.

In [79]:
for item in encoded:
    while len(item) < max_len:
        item.append(0)

padded_np = np.array(encoded)

In [80]:
padded_np

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

## ?-2. Tensorflow로 구현하기

실습을 이어서 하기 위해 패딩을 하기 전으로 초기화하겠습니다.  
케라스에서는 위와 같은 패딩을 위한 도구 tf.keras.preprocessing.sequence.pad_sequences()를 제공하고 있습니다.



In [88]:
tkn.fit_on_texts(corpus)

encoded = tkn.texts_to_sequences(corpus)

padded = tf.keras.preprocessing.sequence.pad_sequences(encoded)

In [89]:
padded

array([[ 0,  0,  0,  0,  0,  1,  5],
       [ 0,  0,  0,  0,  1,  8,  5],
       [ 0,  0,  0,  0,  1,  3,  5],
       [ 0,  0,  0,  0,  0,  9,  2],
       [ 0,  0,  0,  2,  4,  3,  2],
       [ 0,  0,  0,  0,  0,  3,  2],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  6],
       [ 0,  0,  0,  0,  1,  4,  2],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 0,  0,  0,  1, 12,  3, 13]])

- Numpy로 패딩을 진행하였을 때와는 패딩 결과가 다른데 그 이유는 tf.keras.preprocessing.sequence.pad_sequences는 기본적으로 문서의 뒤에 0을 채우는 것이 아니라 앞에 0으로 채우기 때문입니다. 뒤에 0을 채우고 싶다면 인자로 padding="post"를 주면됩니다.

In [90]:
padded = tf.keras.preprocessing.sequence.pad_sequences(encoded, padding = "post")

In [91]:
padded 

array([[ 1,  5,  0,  0,  0,  0,  0],
       [ 1,  8,  5,  0,  0,  0,  0],
       [ 1,  3,  5,  0,  0,  0,  0],
       [ 9,  2,  0,  0,  0,  0,  0],
       [ 2,  4,  3,  2,  0,  0,  0],
       [ 3,  2,  0,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  6,  0,  0,  0,  0],
       [ 1,  4,  2,  0,  0,  0,  0],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0,  0,  0]])

- 지금까지는 가장 긴 길이를 가진 문서의 길이를 기준으로 패딩을 한다고 가정하였지만, 실제로는 꼭 가장 긴 문서의 길이를 기준으로 해야하는 것은 아닙니다. 가령, 모든 문서의 평균 길이가 20인데 문서 1개의 길이가 5,000이라고 해서 굳이 모든 문서의 길이를 5,000으로 패딩할 필요는 없을 수 있습니다. 이와 같은 경우에는 길이에 제한을 두고 패딩할 수 있습니다. max_len의 인자로 정수를 주면, 해당 정수로 모든 문서의 길이를 동일하게 합니다.

In [94]:
padded = tf.keras.preprocessing.sequence.pad_sequences(encoded, padding = "post", maxlen = 5)

In [95]:
padded

array([[ 1,  5,  0,  0,  0],
       [ 1,  8,  5,  0,  0],
       [ 1,  3,  5,  0,  0],
       [ 9,  2,  0,  0,  0],
       [ 2,  4,  3,  2,  0],
       [ 3,  2,  0,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  6,  0,  0],
       [ 1,  4,  2,  0,  0],
       [ 3,  2, 10,  1, 11],
       [ 1, 12,  3, 13,  0]])

- 길이가 5보다 짧은 문서들은 0으로 패딩되고, 기존에 5보다 길었다면 데이터가 손실됩니다. 숫자 0으로 패딩하는 것은 널리 퍼진 관례이긴 하지만, 반드시 지켜야하는 규칙은 아닙니다. 만약, 숫자 0이 아니라 다른 숫자를 패딩을 위한 숫자로 사용하고 싶다면 이 또한 가능합니다. 현재 사용된 정수들과 겹치지 않도록, 단어 집합의 크기에 +1을 한 숫자로 사용해봅시다.

In [96]:
last_value = len(tokenizer.word_index) + 1
print(last_value)

14


In [97]:
padded = tf.keras.preprocessing.sequence.pad_sequences(encoded, padding = "post", value = last_value)

In [98]:
padded

array([[ 1,  5, 14, 14, 14, 14, 14],
       [ 1,  8,  5, 14, 14, 14, 14],
       [ 1,  3,  5, 14, 14, 14, 14],
       [ 9,  2, 14, 14, 14, 14, 14],
       [ 2,  4,  3,  2, 14, 14, 14],
       [ 3,  2, 14, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  6, 14, 14, 14, 14],
       [ 1,  4,  2, 14, 14, 14, 14],
       [ 7,  7,  3,  2, 10,  1, 11],
       [ 1, 12,  3, 13, 14, 14, 14]])

# ?. One-Hot Encoding

## ?-1. Python built-in functions로 구현하기
- 토큰을 입력하면 해당 토큰에 대한 원-핫 벡터를 만들어내는 함수를 만들겠습니다.

In [114]:
corpus = okt.morphs("나는 자연어 처리를 배운다")  

print(corpus)

['나', '는', '자연어', '처리', '를', '배운다']


In [115]:
word2idx = {}
for voca in corpus:
     if voca not in word2idx.keys():
        word2idx[voca] = len(word2idx)

In [116]:
print(word2idx)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


In [121]:
def make_ohe(word, word2idx=word2idx):
        ohe = [0]*(len(word2idx))
        idx = word2idx[word]
        ohe[idx] = 1  
        return ohe

## ?-2. Tensorflow로 구현하기

In [145]:
text = "나랑 점심 먹으러 갈래 점심 메뉴는 햄버거 갈래 갈래 햄버거 최고야"

tkn = tf.keras.preprocessing.text.Tokenizer()
tkn.fit_on_texts([text])

In [146]:
print(tkn.word_index)

{'갈래': 1, '점심': 2, '햄버거': 3, '나랑': 4, '먹으러': 5, '메뉴는': 6, '최고야': 7}


- 위와 같이 생성된 vocabulary에 있는 단어들로만 구성된 텍스트가 있다면, `tkn.texts_to_sequences()`를 통해서 이를 정수 시퀀스로 변환가능합니다. 생성된 단어 집합 내의 일부 단어들로만 구성된 서브 텍스트인 sub_text를 만들어 확인해보겠습니다.

In [151]:
sub_text = "점심 먹으러 갈래 메뉴는 햄버거 최고야"

encoded = tkn.texts_to_sequences([sub_text])[0]

In [152]:
encoded

[2, 5, 1, 6, 3, 7]

- 지금까지 진행한 것은 이미 정수 인코딩 챕터에서 배운 내용입니다. 이제 해당 결과를 가지고, 원-핫 인코딩을 진행해보겠습니다. 케라스는 정수 인코딩 된 결과로부터 원-핫 인코딩을 수행하는 tf.keras.utils.to_categorical()를 지원합니다.

In [153]:
one_hot = tf.keras.utils.to_categorical(encoded)

print(one_hot)

[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


- 위의 결과는 "점심 먹으러 갈래 메뉴는 햄버거 최고야"라는 문장이 `[2, 5, 1, 6, 3, 7]`로 정수 인코딩이 되고나서, 각각의 인코딩 된 결과를 인덱스로 원-핫 인코딩이 수행된 모습을 보여줍니다.

In [198]:
twt.add_dictionary("은경이", "Noun")

In [196]:
twt.morphs("은경이는 사무실로 갔습니다.")

['은경이', '는', '사무실', '로', '갔습니다', '.']